In [ ]:
import pandas as pd
import numpy as np
import pickle

# Load clustered data from previous notebook using pickle
with open('step4.pkl', 'rb') as f:
    final_data_bundle = pickle.load(f)

df = final_data_bundle['df']
customer_features = final_data_bundle['customer_features']
X = final_data_bundle['X']
kmeans_model = final_data_bundle['kmeans_model']

print("Clustered data loaded from step4.pkl:", df.shape)
print("Customer features loaded:", customer_features.shape)

# Cluster Analysis & Insights
Profile clusters (e.g., avg spend, frequency), visualize clusters with plots, and derive actionable insights for marketing and personalization.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Profile Clusters

In [ ]:
cluster_summary = df.groupby('cluster').agg({'price': ['mean', 'count'], 'user_id': 'nunique'})
print(cluster_summary)

## Visualize Clusters

In [ ]:
sns.countplot(x='cluster', data=df)
plt.title('Cluster Counts')
plt.show()

## Actionable Insights
Summarize key findings and recommendations for marketing and personalization based on cluster profiles.